In [ ]:
from flask import Flask,jsonify,abort,make_response,request,url_for
import pickle
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.probability import FreqDist
from heapq import nlargest
from collections import defaultdict



import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt') 
import re
from nltk.tokenize import sent_tokenize        #common
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx   # Converting sim_mat into a graph

In [ ]:
app = Flask(__name__)


@app.route("/api",methods=["POST"])
def model_predicts():
    input_data = request.get_json(force=True)
#     data = [[input_data["s1"],input_data["s2"],input_data["s3"],input_data["s4"]]]
#     print(data)
#     data = np.array(data,np.float32)
#     y_pred = mymodel.predict(data)
    
#     y_response = [float(y_pred[0])]
    y_response = mysum(input_data["data"])
    print(y_response)

    return jsonify({'results':y_response})

if __name__=='__main__':
    app.run(port=9000)

In [ ]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new



def myfunction():
    df = request.getjson(forced=True)
    sentences = [sent_tokenize(x) for x in df['message']]
    sentences = [y for x in sentences for y in x] # flatten list
    print("{0} unread chats found with total of {1} sentences in it.".format(df.shape[0], len(sentences)))
    
    # Extract word vectors #common
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    
    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    # remove stopwords from the sentences  #common
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    
    
    sentence_vectors = []
    for i in clean_sentences:
      if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
      else:
        v = np.zeros((100,))
      sentence_vectors.append(v)
    
    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    
    

    for i in range(len(sentences)):
      for j in range(len(sentences)):
        if i != j:
          sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
        
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    
    
    class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

    # Chat Analytics
    print(color.RED+"{0}".format("Atharv")+color.END+" has {0} chats".format(6))
    print(color.RED+"{0}".format("Krunal")+color.END+" has {1} chats".format("Krunal", 16))
    print("\nTotal "+color.GREEN+"{0}".format(22)+color.END+" chat messages received, with {0} participants\n".format(2))
    print(color.BOLD+"Summary:\n"+color.END)
    for i in range(7):
      print(ranked_sentences[i][1])